In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


1.2.1 Classification accuracy (average):

In [ ]:
import sys
import os
path = '/content/drive/My Drive/Scene-Classification'
os.chdir(path)
os.listdir(path)
# sys.path.append('')
# ! python options.py

['config.py',
 'evaluation.py',
 'simple.py',
 'save',
 '__pycache__',
 'logs',
 'utils.py',
 'multi_models.py',
 'Thesis Record.gdoc',
 'Preprocessing_Multimodal.ipynb',
 'Models_for_using.gdoc',
 'scene-classification-images-and-audio.zip',
 'scene_dataset.py',
 'data',
 'sampling.py',
 'unimodal_image_adam_0.001best_checkpoint.pt',
 'options.py',
 'unimodal_image_sgd_0.001best_checkpoint.pt',
 'multimodal_image_audio_sgd_0.001best_checkpoint.pt',
 'multimodal_image_audio_adam_0.0001best_checkpoint.pt',
 'multimodal_image_audio_sgd_0.0001best_checkpoint.pt',
 'Copy of models.py',
 'federated_main_unimodal.py',
 'update.py',
 'submissionAmaniCaioTest1.csv',
 'submissionAmaniCaioTest.csv',
 'get_data.py',
 'federated_main_multimodal.py',
 'models.py',
 'multimodal_image_audio_adam_0.001best_checkpoint.pt',
 'Scene-Classification.ipynb',
 'Evaluation.ipynb']

In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 125 kB 3.9 MB/s 


In [ ]:
import sys
sys.path.append('My Drive/Colab Notebooks/Scene-Classification')
import options
!ls /content/drive/My\ Drive/Scene-Classification/*.py

'/content/drive/My Drive/Scene-Classification/config.py'
'/content/drive/My Drive/Scene-Classification/Copy of models.py'
'/content/drive/My Drive/Scene-Classification/evaluation.py'
'/content/drive/My Drive/Scene-Classification/federated_main_multimodal.py'
'/content/drive/My Drive/Scene-Classification/federated_main_unimodal.py'
'/content/drive/My Drive/Scene-Classification/get_data.py'
'/content/drive/My Drive/Scene-Classification/models.py'
'/content/drive/My Drive/Scene-Classification/multi_models.py'
'/content/drive/My Drive/Scene-Classification/options.py'
'/content/drive/My Drive/Scene-Classification/sampling.py'
'/content/drive/My Drive/Scene-Classification/scene_dataset.py'
'/content/drive/My Drive/Scene-Classification/simple.py'
'/content/drive/My Drive/Scene-Classification/update.py'
'/content/drive/My Drive/Scene-Classification/utils.py'


In [ ]:
import os
import copy
import time
import pickle
import numpy as np
import torch.nn as nn
import torch
from tqdm import tqdm
import torchvision
from tensorboardX import SummaryWriter
from options import args_parser
from update import LocalUpdate, test_inference,multi_test_inference
from models import vgg_16, mlp_fusion
import get_data #, scene_dataset
from scene_dataset import ImageDataset, AudioDataset, ImageAudioDataset
# from utils import get_dataset, average_weights, exp_details
# from videosets import VideoFrameDataset, ImglistToTensor

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import pandas as pd
from pathlib import Path
from sklearn.metrics import f1_score
from sklearn import preprocessing
def dataset_card(condition):
    class_list = {'FOREST': 0, 'CITY': 1, 'BEACH': 2, 'CLASSROOM': 3, 'RIVER': 4, 'JUNGLE': 5, 'RESTAURANT': 6,
                  'GROCERY-STORE': 7, 'FOOTBALL-MATCH': 8}
    datadir = Path.cwd() / 'data'
    if condition == 'image':
        # train data
        data_df_train = pd.read_csv(os.path.join(datadir, 'train.csv'), delimiter=',', nrows=None)
        data_df_train['CLASS2'] = data_df_train['CLASS2'].map(class_list)
        train_data = np.array(data_df_train)
        train_img_list = data_df_train['IMAGE']
        train_labels = train_data[:, -1].astype('int32')
        # test data
        data_df_test = pd.read_csv(os.path.join(datadir, 'test.csv'), delimiter=',', nrows=None)
        data_df_test['CLASS2'] = data_df_test['CLASS2'].map(class_list)
        test_data = np.array(data_df_test)
        test_img_list = data_df_test['IMAGE']
        test_labels = test_data[:, -1].astype('int32')
        train_transform = transforms.Compose([
            transforms.Resize(size=(224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),

        ])
        test_transform = transforms.Compose([
            # transforms.Grayscale(),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        # train_dataset = datasets.ImageFolder('data/CXR/train', transform=train_transform)
        train_dataset = ImageDataset(root_dir=datadir,
                                    files=train_img_list,
                                    labels=train_labels,
                                    img_transform=train_transform)
        test_dataset = ImageDataset(root_dir=datadir,
                                    files=test_img_list,
                                    labels=test_labels,
                                    img_transform=test_transform)

    if condition == 'image_audio':
        # train data
        data_df_train = pd.read_csv(os.path.join(datadir, 'train.csv'), delimiter=',', nrows=None)
        data_df_train['CLASS2'] = data_df_train['CLASS2'].map(class_list)
        train_data = np.array(data_df_train)
        train_img_list = data_df_train['IMAGE']
        train_audio = train_data[:, 1:-2].astype('float32')
        train_labels = train_data[:, -1].astype('int32')
        scaler_train = preprocessing.StandardScaler().fit(train_audio)
        audio_train_scaled = scaler_train.transform(train_audio)
        # test data
        data_df_test = pd.read_csv(os.path.join(datadir, 'test.csv'), delimiter=',', nrows=None)
        data_df_test['CLASS2'] = data_df_test['CLASS2'].map(class_list)
        test_data = np.array(data_df_test)
        test_img_list = data_df_test['IMAGE']
        test_audio = test_data[:, 1:-2].astype('float32')
        test_labels = test_data[:, -1].astype('int32')
        scaler_test = preprocessing.StandardScaler().fit(test_audio)
        audio_test_scaled = scaler_test.transform(test_audio)
        train_transform = transforms.Compose([
            transforms.Resize(size=(224, 224)),
            # transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
        test_transform = transforms.Compose([
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
        audio_transform = None
        train_dataset = ImageAudioDataset(root_dir=datadir,
                                          files=train_img_list,
                                          audio=audio_train_scaled,
                                          labels=train_labels,
                                          img_transform=train_transform,
                                          audio_transform=audio_transform)
        test_dataset = ImageAudioDataset(root_dir=datadir,
                                          files=test_img_list,
                                          audio=audio_test_scaled,
                                          labels=test_labels,
                                          img_transform=test_transform,
                                          audio_transform=audio_transform)
    #     test_dataset = datasets.ImageFolder('data/CT/test', transform=test_transform)
        # vali_dataset = datasets.ImageFolder('data/CT/vali', transform=test_transform)
        # train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=1)
        # test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=1)
    return train_dataset, test_dataset

# def test_inference(model, test_dataset):
#     """
#     Returns the test accuracy and loss.
#     """
#     model.eval()
#     with torch.no_grad():
#         loss, total, correct = 0.0, 0.0, 0.0
#         pred_list = []
#         y_true = []
#         # device = 'cuda' if args.gpu else 'cpu'
#         device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
#         criterion = nn.CrossEntropyLoss().to(device)
#         testloader = DataLoader(test_dataset, batch_size=128, shuffle=False)
#         for batch_idx, (images, labels) in enumerate(testloader):
#             images, labels = images.to(device), labels.to(device)
#             # Inference
#             outputs = model(images)
#             batch_loss = criterion(outputs, labels)
#             loss += batch_loss.item()
#             # Prediction
#             _, pred_labels = torch.max(outputs, 1)
#             pred_labels = pred_labels.view(-1)
#             correct += torch.sum(torch.eq(pred_labels, labels)).item()
#             total += len(labels)
#             pred_list.extend(pred_labels.cpu().numpy())
#             y_true.extend(labels.cpu().numpy())

#         accuracy = correct / total
#         loss = loss / len(testloader)
#         f1 = f1_score(y_true, pred_list, average=None)
#     return accuracy, loss, f1



### Image modality for nine classes
Train Accruray: 0.944859068183465 \\
Loss: 0.17766076612665696 \\
[0.97870412 0.91946993 0.99008711 0.97593097 0.91272727 0.97732325
 0.87192324 0.92902833 0.94380231] 0.9443329490326295 \\
 ------------------------------------ \\
Test Accuracy: 0.9327536231884058 \\
Loss: 0.22084674558469228 \\
[0.97794118 0.89878543 0.98806683 0.96350365 0.8960396  0.97183099
 0.86378738 0.90167866 0.93890675] 0.9333933835288425
### Audio modality for nine classes
Train Accruray: 0.9704369248605174 \\
Loss: 0.11190520517131383 \\
[0.99493321 0.96381151 0.9740566  0.97456857 0.96302976 0.99866607
 0.93766012 0.9740376  0.95762376] 0.9709319120795106 \\
--------------------------------------- \\
Test Accuracy: 0.936231884057971 \\
Loss: 0.2986410068614142 \\
[0.98141264 0.93697479 0.94859813 0.92673993 0.91262136 1.
 0.83687943 0.94009217 0.95327103] 0.9373988302984038

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = vgg_16(in_channels=3, num_classes=9)
model.to(device)
PATH_pth = '/content/drive/My Drive/Scene-Classification/unimodal_image_adam_0.001best_checkpoint.pt'
state_dict = torch.load(PATH_pth)
model.load_state_dict(state_dict, strict=False)
train_dataset, test_dataset = dataset_card('image')
accuracy_train, loss_train, f1_train = test_inference(model, train_dataset)
print(accuracy_train)
print(loss_train)
print(f1_train, sum(f1_train)/len(f1_train))
print('------------------------------------')
accuracy_test, loss_test, f1_test = test_inference(model, test_dataset)
print(accuracy_test)
print(loss_test)
print(f1_test, sum(f1_test)/len(f1_test))

0.944859068183465
0.17766076612665696
[0.97870412 0.91946993 0.99008711 0.97593097 0.91272727 0.97732325
 0.87192324 0.92902833 0.94380231] 0.9443329490326295
------------------------------------
0.9327536231884058
0.22084674558469228
[0.97794118 0.89878543 0.98806683 0.96350365 0.8960396  0.97183099
 0.86378738 0.90167866 0.93890675] 0.9333933835288425


### Audio modality for nine classes


In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = mlp_fusion()
model.to(device)
PATH_pth = '/content/drive/My Drive/Scene-Classification/multimodal_image_audio_adam_0.001best_checkpoint.pt'
state_dict = torch.load(PATH_pth)
model.load_state_dict(state_dict, strict=False)
train_dataset, test_dataset = dataset_card('image_audio')
accuracy_train, loss_train, f1_train = multi_test_inference(model, train_dataset)
print(accuracy_train)
print(loss_train)
print(f1_train, sum(f1_train)/len(f1_train))
print('------------------------------------')
accuracy_test, loss_test, f1_test = multi_test_inference(model, test_dataset)
print(accuracy_test)
print(loss_test)
print(f1_test, sum(f1_test)/len(f1_test))

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

0.9706543004130136
0.10956076052281316
[0.99357208 0.96193416 0.9803805  0.97696477 0.96685578 0.9973357
 0.92828861 0.96989568 0.96063618] 0.970651495079777
------------------------------------
0.9350724637681159
0.2715472163898604
[0.98540146 0.93004115 0.95486936 0.93430657 0.92344498 0.99646643
 0.82962963 0.92807425 0.93416928] 0.9351559002076022
